In [1]:
# -*- coding: utf-8 -*-

from PyQt5 import QtCore, QtGui, QtWidgets
from mplwidget import MplWidget
from matplotlib.backends.backend_qt5agg import (NavigationToolbar2QT as NavigationToolbar)


from seabreeze.spectrometers import Spectrometer
from matplotlib import pyplot as plt
import time
import numpy as np
import serial
import sys
import numpy as np
import csv
sys.path.insert(1, '..')
from scan_functions import * # functions built for this project
import pandas as pd
import sqlite3
import datetime
from scipy.signal import savgol_filter as sgf
from math import factorial as fact
from tkinter import Tk, filedialog
from scipy.signal import find_peaks as peaks
from IPython.display import clear_output
  

class Ui_experimentWindow(QtWidgets.QMainWindow):
    
    def __init__(self, parent = None):
        super().__init__()
        icon = QtGui.QIcon()
        icon.addPixmap(QtGui.QPixmap("web.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        self.setWindowIcon(icon)    
        self.conn = sqlite3.connect('../Database/Sniffing-Sensor.db') # connect to the database
        self.c = self.conn.cursor() # Creates a cursor to interact with database
        self.c.execute("SELECT ExperimentID FROM Experiments")
        self.data = self.c.fetchall()
        self.exp_name = "Default"
        if self.data == []: 
            self.exp_name = 1
        else:
            self.exp_name = self.data[-1][0] + 1
        
        print(self.exp_name)
        
        self.expThread = expThread
       
    def defaultComm(self): 
        self.teensyInput.setText("4")
        self.integrationInput.setText("7")
        print("hit default comm")
        self.serConnect("4")
        self.spec_settings("7")
        
    def defaultName(self): 
        self.unameInput.setText("Ethan Thompson")
        self.username()
        
    def username(self): 
        username = self.unameInput.text()
        print("Bleh ufas")
        self.c.execute(""" """)
    
    def spec_settings(self, integration): 
        try: 
            print(integration)
            self.spec = Spectrometer.from_first_available() # Start communication with spectrometer
            self.time_in_ms = int(integration.strip())*10**3
            self.spec.integration_time_micros(self.time_in_ms) # Sets integration time
        except:
            print("Can't connect to spectrometer")
    
    def serConnect(self, port): 
        try:   
            port = port.strip()
            print(port)
            self.ser = serial.Serial("COM{}".format(port), 9600) # Opening serial port to communicate with teensy
        except: 
            print("Can't connect to teensy")
    
    def file_open(self): 
        fname = QtWidgets.QFileDialog.getOpenFileName(self, 'Open file', '../..',"Excel Files (*.xlsx)")
        self.experiment = pd.read_excel('{}'.format(fname[0]))
        self.experiment.set_index("True Runs", inplace=True) # indexed by the True run number
        self.experiment.dropna(axis=0, how="all", inplace=True) #removes all NaN rows
        self.uploadFile.setText("%s"%(fname[0].split("/")[-1]))
        
        print(self.experiment.head())
        
    def run_exp(self): 
        self.expThread.run_exp(self)
        print("who's there?")
    
    def setupUi(self, experimentWindow):
        
        experimentWindow.setObjectName("experimentWindow")
        experimentWindow.resize(1229, 874)
        icon = QtGui.QIcon()
        icon.addPixmap(QtGui.QPixmap("web.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        experimentWindow.setWindowIcon(icon)
        self.centralWidget = QtWidgets.QWidget(experimentWindow)
        self.centralWidget.setObjectName("centralWidget")
        self.userScroll = QtWidgets.QScrollArea(self.centralWidget)
        self.userScroll.setGeometry(QtCore.QRect(20, 30, 435, 781))
        self.userScroll.setMaximumSize(QtCore.QSize(435, 16777215))
        self.userScroll.setWidgetResizable(True)
        self.userScroll.setObjectName("userScroll")
        self.userInput = QtWidgets.QWidget()
        self.userInput.setGeometry(QtCore.QRect(0, 0, 433, 779))
        self.userInput.setObjectName("userInput")
        self.settingsFrame = QtWidgets.QGroupBox(self.userInput)
        self.settingsFrame.setGeometry(QtCore.QRect(10, 10, 400, 119))
        self.settingsFrame.setMinimumSize(QtCore.QSize(400, 100))
        self.settingsFrame.setMaximumSize(QtCore.QSize(16777215, 400))
        self.settingsFrame.setObjectName("settingsFrame")
        self.verticalLayout_2 = QtWidgets.QVBoxLayout(self.settingsFrame)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.settingsInput = QtWidgets.QGridLayout()
        self.settingsInput.setObjectName("settingsInput")
        
        # username box
        self.unameInput = QtWidgets.QLineEdit(self.settingsFrame)
        self.unameInput.setInputMask("")
        self.unameInput.setText("")
        self.unameInput.setFrame(True)
        self.unameInput.setCursorPosition(0)
        self.unameInput.setReadOnly(False)
        self.unameInput.setObjectName("unameInput")
        self.unameInput.returnPressed.connect(self.username)
        
        self.settingsInput.addWidget(self.unameInput, 0, 1, 1, 1)
        self.expNum = QtWidgets.QLabel(self.settingsFrame)
        self.expNum.setMaximumSize(QtCore.QSize(16777215, 10))
        
        self.expNum.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.expNum.setObjectName("expNum")
        self.settingsInput.addWidget(self.expNum, 1, 0, 1, 1)
        self.unameLabel = QtWidgets.QLabel(self.settingsFrame)
        self.unameLabel.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.unameLabel.setObjectName("unameLabel")
        self.settingsInput.addWidget(self.unameLabel, 0, 0, 1, 1)
        self.label_13 = QtWidgets.QLabel(self.settingsFrame)
        self.label_13.setAlignment(QtCore.Qt.AlignCenter)
        self.label_13.setObjectName("label_13")
        self.settingsInput.addWidget(self.label_13, 1, 1, 1, 1)
        
        
        self.defaultCheck = QtWidgets.QCheckBox(self.settingsFrame)
        self.defaultCheck.setObjectName("defaultCheck")
        self.defaultCheck.clicked.connect(self.defaultName)
        
        self.settingsInput.addWidget(self.defaultCheck, 2, 1, 1, 1)
        self.verticalLayout_2.addLayout(self.settingsInput)
        self.commFrame = QtWidgets.QGroupBox(self.userInput)
        self.commFrame.setGeometry(QtCore.QRect(10, 130, 400, 182))
        self.commFrame.setMinimumSize(QtCore.QSize(400, 100))
        self.commFrame.setMaximumSize(QtCore.QSize(16777215, 400))
        self.commFrame.setObjectName("commFrame")
        self.verticalLayout_4 = QtWidgets.QVBoxLayout(self.commFrame)
        self.verticalLayout_4.setObjectName("verticalLayout_4")
        self.commInput = QtWidgets.QGridLayout()
        self.commInput.setObjectName("commInput")
        
        self.teensyInput = QtWidgets.QLineEdit(self.commFrame)
        self.teensyInput.setInputMask("")
        self.teensyInput.setText("")
        self.teensyInput.setFrame(True)
        self.teensyInput.setCursorPosition(0)
        self.teensyInput.setReadOnly(False)
        self.teensyInput.setObjectName("teensyInput")
        self.teensyInput.returnPressed.connect(lambda: self.serConnect(self.teensyInput.text()))
        
        self.commInput.addWidget(self.teensyInput, 0, 1, 1, 1)
        self.integration = QtWidgets.QLabel(self.commFrame)
        self.integration.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.integration.setObjectName("integration")
        self.commInput.addWidget(self.integration, 1, 0, 1, 1)
        
        self.checkBox_2 = QtWidgets.QCheckBox(self.commFrame)
        self.checkBox_2.setObjectName("checkBox_2")
        self.checkBox_2.clicked.connect(self.defaultComm)
        
        self.commInput.addWidget(self.checkBox_2, 2, 1, 1, 1)
               
        self.integrationInput = QtWidgets.QLineEdit(self.commFrame)
        self.integrationInput.setInputMask("")
        self.integrationInput.setText("")
        self.integrationInput.setFrame(True)
        self.integrationInput.setCursorPosition(0)
        self.integrationInput.setReadOnly(False)
        self.integrationInput.setObjectName("integrationInput")
        self.integrationInput.returnPressed.connect(lambda: self.spec_settings(self.integrationInput.text()))
        
        self.commInput.addWidget(self.integrationInput, 1, 1, 1, 1)
        
        self.teensyPort = QtWidgets.QLabel(self.commFrame)
        self.teensyPort.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.teensyPort.setObjectName("teensyPort")
        
        self.commInput.addWidget(self.teensyPort, 0, 0, 1, 1)
        
        self.verticalLayout_4.addLayout(self.commInput)
        self.line1 = QtWidgets.QFrame(self.userInput)
        self.line1.setGeometry(QtCore.QRect(50, 320, 319, 3))
        self.line1.setFrameShape(QtWidgets.QFrame.HLine)
        self.line1.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.line1.setObjectName("line1")
        
        self.uploadFile = QtWidgets.QCommandLinkButton(self.userInput)
        self.uploadFile.setEnabled(True)
        self.uploadFile.setGeometry(QtCore.QRect(50, 320, 319, 50))
        self.uploadFile.setMaximumSize(QtCore.QSize(16777215, 50))
        self.uploadFile.setCursor(QtGui.QCursor(QtCore.Qt.ArrowCursor))
        self.uploadFile.setAutoFillBackground(False)
        self.uploadFile.setAutoDefault(False)
        self.uploadFile.setObjectName("uploadFile")
        self.uploadFile.clicked.connect(self.file_open)
        
        self.line3 = QtWidgets.QFrame(self.userInput)
        self.line3.setGeometry(QtCore.QRect(40, 450, 319, 3))
        self.line3.setFrameShape(QtWidgets.QFrame.HLine)
        self.line3.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.line3.setObjectName("line3")
        
        self.startButton = QtWidgets.QPushButton(self.userInput)
        self.startButton.setGeometry(QtCore.QRect(50, 370, 319, 28))
        self.startButton.setObjectName("startButton")
        self.progressBar = QtWidgets.QProgressBar(self.userInput)
        self.progressBar.setGeometry(QtCore.QRect(50, 400, 319, 24))
        self.progressBar.setProperty("value", 24)
        self.progressBar.setAlignment(QtCore.Qt.AlignCenter)
        self.progressBar.setObjectName("progressBar")
        self.line4 = QtWidgets.QFrame(self.userInput)
        self.line4.setGeometry(QtCore.QRect(40, 540, 319, 20))
        self.line4.setFrameShape(QtWidgets.QFrame.HLine)
        self.line4.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.line4.setObjectName("line4")
        self.saveBox = QtWidgets.QGroupBox(self.userInput)
        self.saveBox.setGeometry(QtCore.QRect(10, 560, 401, 211))
        self.saveBox.setObjectName("saveBox")
        self.gridLayoutWidget = QtWidgets.QWidget(self.saveBox)
        self.gridLayoutWidget.setGeometry(QtCore.QRect(40, 30, 271, 135))
        self.gridLayoutWidget.setObjectName("gridLayoutWidget")
        self.saveGrid = QtWidgets.QGridLayout(self.gridLayoutWidget)
        self.saveGrid.setContentsMargins(0, 0, 0, 0)
        self.saveGrid.setObjectName("saveGrid")
        self.phaseCheck = QtWidgets.QCheckBox(self.gridLayoutWidget)
        self.phaseCheck.setObjectName("phaseCheck")
        self.saveGrid.addWidget(self.phaseCheck, 2, 1, 1, 1)
        self.fourierCheck = QtWidgets.QCheckBox(self.gridLayoutWidget)
        self.fourierCheck.setObjectName("fourierCheck")
        self.saveGrid.addWidget(self.fourierCheck, 1, 1, 1, 1)
        self.pdCheck = QtWidgets.QCheckBox(self.gridLayoutWidget)
        self.pdCheck.setObjectName("pdCheck")
        self.saveGrid.addWidget(self.pdCheck, 3, 1, 1, 1)
        self.riverCheck = QtWidgets.QCheckBox(self.gridLayoutWidget)
        self.riverCheck.setObjectName("riverCheck")
        self.saveGrid.addWidget(self.riverCheck, 0, 1, 1, 1)
        self.riverLoc = QtWidgets.QPushButton(self.gridLayoutWidget)
        self.riverLoc.setObjectName("riverLoc")
        self.saveGrid.addWidget(self.riverLoc, 0, 0, 1, 1)
        self.fourierLoc = QtWidgets.QPushButton(self.gridLayoutWidget)
        self.fourierLoc.setObjectName("fourierLoc")
        self.saveGrid.addWidget(self.fourierLoc, 1, 0, 1, 1)
        self.phaseLoc = QtWidgets.QPushButton(self.gridLayoutWidget)
        self.phaseLoc.setObjectName("phaseLoc")
        self.saveGrid.addWidget(self.phaseLoc, 2, 0, 1, 1)
        self.pdLoc = QtWidgets.QPushButton(self.gridLayoutWidget)
        self.pdLoc.setObjectName("pdLoc")
        self.saveGrid.addWidget(self.pdLoc, 3, 0, 1, 1)
        self.saveCheck = QtWidgets.QCheckBox(self.saveBox)
        self.saveCheck.setGeometry(QtCore.QRect(20, 180, 181, 20))
        self.saveCheck.setObjectName("saveCheck")
        self.progressText = QtWidgets.QLabel(self.userInput)
        self.progressText.setGeometry(QtCore.QRect(40, 460, 311, 91))
        self.progressText.setAutoFillBackground(False)
        self.progressText.setText("")
        
        self.progressText.setAlignment(QtCore.Qt.AlignLeading|QtCore.Qt.AlignLeft|QtCore.Qt.AlignTop)
        self.progressText.setObjectName("progressText")
        self.userScroll.setWidget(self.userInput)
        
#         self.startButton = QtWidgets.QPushButton(self.userInput)
#         self.startButton.setGeometry(QtCore.QRect(50, 610, 319, 28))
#         self.startButton.setObjectName("startButton")
#         self.startButton.clicked.connect(self.run_exp)
        
#         self.progressBar = QtWidgets.QProgressBar(self.userInput)
#         self.progressBar.setGeometry(QtCore.QRect(50, 640, 319, 24))
#         self.progressBar.setProperty("value", 24)
#         self.progressBar.setAlignment(QtCore.Qt.AlignCenter)
#         self.progressBar.setObjectName("progressBar")
        
#         self.line4 = QtWidgets.QFrame(self.userInput)
#         self.line4.setGeometry(QtCore.QRect(50, 670, 319, 3))
#         self.line4.setFrameShape(QtWidgets.QFrame.HLine)
#         self.line4.setFrameShadow(QtWidgets.QFrame.Sunken)
#         self.line4.setObjectName("line4")
#         self.visibleCheck = QtWidgets.QCheckBox(self.userInput)
#         self.visibleCheck.setGeometry(QtCore.QRect(140, 680, 153, 20))
#         self.visibleCheck.setObjectName("visibleCheck")
#         self.userScroll.setWidget(self.userInput)

        self.plotFrame = QtWidgets.QFrame(self.centralWidget)
        self.plotFrame.setGeometry(QtCore.QRect(460, 30, 1040, 781))
        self.plotFrame.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.plotFrame.setFrameShadow(QtWidgets.QFrame.Raised)
        self.plotFrame.setObjectName("plotFrame")
        self.tabs = QtWidgets.QTabWidget(self.plotFrame)
        self.tabs.setGeometry(QtCore.QRect(0, 0, 761, 781))
        self.tabs.setMaximumSize(QtCore.QSize(1000, 16777215))
        self.tabs.setObjectName("tabs")
        self.riverTab = QtWidgets.QWidget()
        self.riverTab.setObjectName("riverTab")
        
        self.mplRiver = MplWidget(self.riverTab)
        self.mplRiver.setGeometry(QtCore.QRect(0, 0, 761, 761))
        self.mplRiver.setMinimumSize(QtCore.QSize(500, 500))
        self.mplRiver.setAutoFillBackground(True)
        self.mplRiver.setObjectName("mplRiver")
        
        self.tabs.addTab(self.riverTab, "")
        self.fourierTab = QtWidgets.QWidget()
        self.fourierTab.setObjectName("fourierTab")
        
        self.mplFourier = MplWidget(self.fourierTab)
        self.mplFourier.setGeometry(QtCore.QRect(0, 0, 761, 761))
        self.mplFourier.setMinimumSize(QtCore.QSize(500, 500))
        self.mplFourier.setAutoFillBackground(True)
        self.mplFourier.setObjectName("mplFourier")
        
        self.tabs.addTab(self.fourierTab, "")
        self.phaseTab = QtWidgets.QWidget()
        self.phaseTab.setObjectName("phaseTab")
        
        self.mplPhase = MplWidget(self.phaseTab)
        self.mplPhase.setGeometry(QtCore.QRect(0, 0, 761, 761))
        self.mplPhase.setMinimumSize(QtCore.QSize(500, 500))
        self.mplPhase.setAutoFillBackground(True)
        self.mplPhase.setObjectName("mplPhase")
        
        self.tabs.addTab(self.phaseTab, "")
        self.pdTab = QtWidgets.QWidget()
        self.pdTab.setObjectName("pdTab")
        
        self.mplPder = MplWidget(self.pdTab)
        self.mplPder.setGeometry(QtCore.QRect(0, 0, 761, 761))
        self.mplPder.setMinimumSize(QtCore.QSize(500, 500))
        self.mplPder.setAutoFillBackground(True)
        self.mplPder.setObjectName("mplPder")
        
        self.tabs.addTab(self.pdTab, "")
        self.steadyTab = QtWidgets.QWidget()
        self.steadyTab.setObjectName("steadyTab")
        
        self.mplSteady = MplWidget(self.steadyTab)
        self.mplSteady.setGeometry(QtCore.QRect(0, 0, 761, 761))
        self.mplSteady.setMinimumSize(QtCore.QSize(500, 500))
        self.mplSteady.setAutoFillBackground(True)
        self.mplSteady.setObjectName("mplSteady")
        
        self.tabs.addTab(self.steadyTab, "")
        experimentWindow.setCentralWidget(self.centralWidget)
        self.menubar = QtWidgets.QMenuBar(experimentWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 1131, 26))
        self.menubar.setObjectName("menubar")
        self.menuFile = QtWidgets.QMenu(self.menubar)
        self.menuFile.setObjectName("menuFile")
        self.menuUndo = QtWidgets.QMenu(self.menubar)
        self.menuUndo.setObjectName("menuUndo")
        self.menuView = QtWidgets.QMenu(self.menubar)
        self.menuView.setObjectName("menuView")
        experimentWindow.setMenuBar(self.menubar)
        self.actionSave = QtWidgets.QAction(experimentWindow)
        self.actionSave.setObjectName("actionSave")
        self.actionExit = QtWidgets.QAction(experimentWindow)
        self.actionExit.setObjectName("actionExit")
        self.actionUndo = QtWidgets.QAction(experimentWindow)
        self.actionUndo.setObjectName("actionUndo")
        self.actionRedo = QtWidgets.QAction(experimentWindow)
        self.actionRedo.setObjectName("actionRedo")
        self.actionPreferences = QtWidgets.QAction(experimentWindow)
        self.actionPreferences.setObjectName("actionPreferences")
        self.actionPreferences_2 = QtWidgets.QAction(experimentWindow)
        self.actionPreferences_2.setObjectName("actionPreferences_2")
        self.actionFullscreen = QtWidgets.QAction(experimentWindow)
        self.actionFullscreen.setObjectName("actionFullscreen")
        self.actionMinimize = QtWidgets.QAction(experimentWindow)
        self.actionMinimize.setObjectName("actionMinimize")
        self.menuFile.addAction(self.actionSave)
        self.menuFile.addSeparator()
        self.menuFile.addAction(self.actionExit)
        self.menuUndo.addAction(self.actionUndo)
        self.menuUndo.addAction(self.actionRedo)
        self.menuUndo.addSeparator()
        self.menuView.addAction(self.actionFullscreen)
        self.menuView.addAction(self.actionMinimize)
        self.menubar.addAction(self.menuFile.menuAction())
        self.menubar.addAction(self.menuUndo.menuAction())
        self.menubar.addAction(self.menuView.menuAction())

        self.retranslateUi(experimentWindow)
        self.tabs.setCurrentIndex(4)
        QtCore.QMetaObject.connectSlotsByName(experimentWindow)

    def retranslateUi(self, experimentWindow):
        _translate = QtCore.QCoreApplication.translate
        experimentWindow.setWindowTitle(_translate("experimentWindow", "Sniffing Sensor 3000"))
        self.settingsFrame.setStatusTip(_translate("experimentWindow", "Experiment settings"))
        self.settingsFrame.setTitle(_translate("experimentWindow", "Experiment Settings:"))
        self.unameInput.setStatusTip(_translate("experimentWindow", "Input user name"))
        self.unameInput.setPlaceholderText(_translate("experimentWindow", "Ethan Thompson"))
        self.expNum.setStatusTip(_translate("experimentWindow", "Set experiment number"))
        self.expNum.setText(_translate("experimentWindow", "Experiment #:"))
        self.unameLabel.setText(_translate("experimentWindow", "User Name:"))
        self.label_13.setStatusTip(_translate("experimentWindow", "Experiment number"))
        self.label_13.setText(_translate("experimentWindow", "{}".format(self.exp_name)))
        self.defaultCheck.setStatusTip(_translate("experimentWindow", "Check for default options"))
        self.defaultCheck.setText(_translate("experimentWindow", "Check for default"))
        self.commFrame.setStatusTip(_translate("experimentWindow", "Communication Settings"))
        self.commFrame.setTitle(_translate("experimentWindow", "Communication Settings:"))
        self.teensyInput.setStatusTip(_translate("experimentWindow", "Input user name"))
        self.teensyInput.setPlaceholderText(_translate("experimentWindow", "4"))
        self.integration.setText(_translate("experimentWindow", "Integration Time:"))
        self.teensyPort.setText(_translate("experimentWindow", "Teensy Port #:"))
        self.checkBox_2.setStatusTip(_translate("experimentWindow", "Check for default"))
        self.checkBox_2.setText(_translate("experimentWindow", "Check for default"))
        self.integrationInput.setStatusTip(_translate("experimentWindow", "Input user name"))
        self.integrationInput.setPlaceholderText(_translate("experimentWindow", "7"))
        self.uploadFile.setStatusTip(_translate("experimentWindow", "Upload an excel file with experiment parameters"))
        self.uploadFile.setText(_translate("experimentWindow", "Upload Experiment File"))
#         self.odorantFrame.setStatusTip(_translate("experimentWindow", "Odorant frame"))
#         self.odorantFrame.setTitle(_translate("experimentWindow", "Set Custom Odorant Cycle:"))
#         self.timeUnit.setText(_translate("experimentWindow", "times"))
#         self.seconds3.setText(_translate("experimentWindow", "seconds"))
#         self.seconds4.setText(_translate("experimentWindow", "seconds"))
#         self.seconds2.setText(_translate("experimentWindow", "seconds"))
#         self.odorant2.setText(_translate("experimentWindow", "Odorant 2:"))
#         self.repeat.setText(_translate("experimentWindow", "Repeat: "))
#         self.odorant3.setText(_translate("experimentWindow", "Odorant 3:"))
#         self.seconds1.setText(_translate("experimentWindow", "seconds"))
#         self.odorant4.setText(_translate("experimentWindow", "Odorant 4:"))
#         self.odorant1.setText(_translate("experimentWindow", "Odorant 1:"))
#         self.addCycle.setStatusTip(_translate("experimentWindow", "Add another cycle"))
#         self.addCycle.setText(_translate("experimentWindow", "Add Another Cycle"))
        
        self.startButton.setStatusTip(_translate("experimentWindow", "Start running the experiment"))
        self.startButton.setText(_translate("experimenWindow", "Start Experiment"))
#         self.visibleCheck.setStatusTip(_translate("experimentWindow", "Suppress plot visibilities and save instead."))
#         self.visibleCheck.setText(_translate("experimentWindow", "Suppress Plot visibility"))
        
        self.saveBox.setTitle(_translate("experimentWindow", "Save CSV Files"))
        self.phaseCheck.setText(_translate("experimentWindow", "Phase"))
        self.fourierCheck.setText(_translate("experimentWindow", "Fourier"))
        self.pdCheck.setText(_translate("experimentWindow", "Phase Derivative"))
        self.riverCheck.setText(_translate("experimentWindow", "River Plot"))
        self.riverLoc.setText(_translate("experimentWindow", "PushButton"))
        self.fourierLoc.setText(_translate("experimentWindow", "PushButton"))
        self.phaseLoc.setText(_translate("experimentWindow", "PushButton"))
        self.pdLoc.setText(_translate("experimentWindow", "PushButton"))
        self.saveCheck.setText(_translate("experimentWindow", "Check for defaults"))    
    
        self.plotFrame.setAccessibleName(_translate("experimentWindow", "Plot Frame"))
        self.plotFrame.setAccessibleDescription(_translate("experimentWindow", "Frame containing tabs for experiment plots"))
        self.tabs.setStatusTip(_translate("experimentWindow", "Plot tabs"))
        self.riverTab.setStatusTip(_translate("experimentWindow", "Plotting River Plot"))
        self.tabs.setTabText(self.tabs.indexOf(self.riverTab), _translate("experimentWindow", "River Plot"))
        self.fourierTab.setStatusTip(_translate("experimentWindow", "Plotting first frequency of fourier transfrom"))
        self.tabs.setTabText(self.tabs.indexOf(self.fourierTab), _translate("experimentWindow", "Fourier"))
        self.phaseTab.setStatusTip(_translate("experimentWindow", "Plotting phase of spectrum"))
        self.tabs.setTabText(self.tabs.indexOf(self.phaseTab), _translate("experimentWindow", "Phase"))
        self.pdTab.setStatusTip(_translate("experimentWindow", "Plotting phase derivative"))
        self.tabs.setTabText(self.tabs.indexOf(self.pdTab), _translate("experimentWindow", "Phase Derivative"))
        self.steadyTab.setStatusTip(_translate("experimentWindow", "Plotting steady state condition and peaks"))
        self.tabs.setTabText(self.tabs.indexOf(self.steadyTab), _translate("experimentWindow", "Steady State"))
        self.menuFile.setTitle(_translate("experimentWindow", "File"))
        self.menuUndo.setTitle(_translate("experimentWindow", "Edit"))
        self.menuView.setTitle(_translate("experimentWindow", "View"))
        self.actionSave.setText(_translate("experimentWindow", "Save"))
        self.actionExit.setText(_translate("experimentWindow", "Exit"))
        self.actionUndo.setText(_translate("experimentWindow", "Undo"))
        self.actionRedo.setText(_translate("experimentWindow", "Redo"))
        self.actionPreferences.setText(_translate("experimentWindow", "Preferences"))
        self.actionPreferences_2.setText(_translate("experimentWindow", "Preferences"))
        self.actionFullscreen.setText(_translate("experimentWindow", "Fullscreen"))
        self.actionMinimize.setText(_translate("experimentWindow", "Minimize"))
        
        
class expThread(QtCore.QThread, Ui_experimentWindow): 
    def __init__(self, level=None, *args, **kwargs):
        super(expThread, self).__init__(*args, **kwargs)

    def run_exp(self): 
        print("creatine")
        to_df = {"Run": [], 
         "Steady State": [], 
         "Peak 1": [], 
         "Peak 2": [], 
         "Peak 3": [], 
         "Trough 1": [], 
         "Trough 2": []}

        df2 = pd.DataFrame(to_df)
        df2.set_index('Run', inplace=True)
        df2.to_excel('../../Experiment Files/{}.xlsx'.format(self.exp_name))
        
        
        # redefining instance variables
        conn = self.conn
        c = self.c
        username = self.username
        spec = self.spec
        time_in_ms = self.time_in_ms
        ser = self.ser        
        experiment = self.experiment
                
        num_exp = len(experiment.index) # create an entry for each experiment run
        
        for run in range(num_exp): 
            clear_output() # clears output after every run

            # --------------- DESIGNING RECIPE -------------------

            o_time = list(experiment['Odorant Time'])[run]
            odorant_time = [o_time for i in range(3)]

            purge = int(list(experiment['Purge time'])[run]) # by default dataframes are floats

            c_time = list(experiment['Sleep Time'])[run]
            clean_time = [purge] + [c_time for i in range(2)]

            delay_time = 10 # scans per second

            # --------------- PERFORMING RECIPE -------------------

            # Experiment Name - Integer ID
            exp_name = self.exp_name

            # Timestamp
            timestamp = str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

            # Compound name coded into its OdorID from the database
            compound = list(experiment['Current'])[run]
            temp = c.execute("SELECT OdorID FROM Odors WHERE Name='%s'" %str(compound.upper())) # All database entries are uppercase
            OdorID = c.fetchall()[0][0]
            print("Run Number: {}\nExperiment Name: {}\nCompound: {}".format(run+1, exp_name, compound))

            # Procedure name coded into its ProcedureID from the database
            temp = c.execute("SELECT ProcedureID FROM Procedures WHERE Name='Testing'")
            ProcedureID = c.fetchall()[0][0]

            # Creates a PNG and CSV File pointing to the location of the files
            PNG = "../Scan CSV Files/{}.png".format(exp_name)
            CSV = "../Scan CSV Files/PNG/{}.png".format(exp_name)


            # ======================= WRITING TO CSV File =======================

            df = pd.DataFrame() # creates a new dataframe for csv file
            df["Wavelengths"] = spec.wavelengths()
            df.set_index("Wavelengths")

            # ============================== RECIPE ==============================

            intensities = [spec.intensities()] # empty list to be populated with intensity values from spec
            elapse = [0] #empty list to be populated with time values

            compound_dict = {"Nitrogen": Nitrogen, 
                             "Water": valveC, 
                             "IPA": valveD, 
                             "Ethanol": valveB,
                             "Pentane": valveC,
                             "Hexane": valveD, 
                             "Heptane": valveB} # Assigns the compound to its injection function

            # If there is a nitrogen flush specified beforehand, then do a 10 minute flush.
            if list(experiment['Preceeding'])[run] == "Nitrogen":
                print("Purging chamber for 10 minutes. Please standby.")
                nitro_arg = [600, ser, intensities, spec, elapse, True]
                Nitrogen(*nitro_arg)

            print("Performing {} injection".format(compound))
            for i in range(len(odorant_time)):
                nitro_arg = [clean_time[i], ser, intensities, spec, elapse]
                Nitrogen(*nitro_arg)

                odorant_arg = [odorant_time[i], ser, intensities, spec, delay_time, elapse]
                compound_dict["{}".format(compound)](*odorant_arg) # Change the function based on what odorant you are testing

            for i in range(len(elapse)):
                df["{}".format(round(elapse[i],2))] = intensities[i] # Write to the i'th column the intensities

            sys.stdout.write("\n{}\nDone Captures!\n".format('-'*30))

            # ========================== WRITING TO FILES ==========================
            print("Saving to CSV in ../../Scan CSV Files/{}.csv".format(exp_name))
            df.to_csv("../../Scan CSV Files/{}.csv".format(exp_name)) # Writing to the CSV file

            Notes = "Testing automation."
            Notes += "\nOdorant Times: {}\nClean Times: {}\n".format(", ".join([str(i) for i in clean_time]),
                                                                     ", ".join([str(i) for i in odorant_time]))

            c.execute("""INSERT INTO Experiments (Timestamp, OdorID, ProcedureID, PNG, CSV, Notes)
                         VALUES (?, ?, ?, ?, ?, ?)""",
                         (timestamp, OdorID, ProcedureID, PNG, CSV, Notes))

            conn.commit() # Save values into SQL

            # ========================== DATA ANALYSIS ==========================
            print("Analyzing data and saving plots.")
            # Read file and convert it to pandas dataframe
            df = pd.read_csv("../../Scan CSV Files/{}.csv".format(exp_name))
            df.drop("Unnamed: 0", axis=1, inplace=True)
            df.set_index("Wavelengths", inplace=True)
            exp_name = compound + "({})".format(exp_name)

            
            ## RIVER PLOT
            sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900
            
            first = float(df.columns[0])
            last = float(df.columns[-1])
            num_ticks = 10
            
            self.addToolBar(NavigationToolbar(self.mplRiver.canvas, self))
            self.mplRiver.canvas.axes.clear()
            self.mplRiver.canvas.axes.imshow(df.transpose(), aspect = len(df.index)/len(df.columns))
            self.mplRiver.canvas.axes.set_title("River Plot")
            self.mplRiver.canvas.axes.set_yticks(np.linspace(0, len(df.columns), num_ticks))
            self.mplRiver.canvas.axes.set_yticklabels(map(int, np.linspace(last, first, num_ticks)))
            self.mplRiver.canvas.axes.set_xlabel('Wavelength (nm)')
            self.mplRiver.canvas.axes.set_ylabel('Time (s)')
            self.mplRiver.canvas.draw_idle()
            

            ## FOURIER TRANSFORm

            # taking the transpose means you get the fourier transform with time axis
            ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

            self.mplFourier.canvas.axes.clear()
            self.mplFourier.canvas.axes.plot(np.real(ft_data[:,1]))
            self.mplFourier.canvas.axes.plot(np.imag(ft_data[:,1]))
            self.mplFourier.canvas.axes.set_title("Fourier Transform")
            self.mplFourier.canvas.axes.set_xticks(np.linspace(0, len(df.columns), num_ticks))
            self.mplFourier.canvas.axes.set_xticklabels(map(int, np.linspace(last, first, num_ticks)))
            self.mplFourier.canvas.axes.set_xlabel('Time (s)')
            self.mplFourier.canvas.axes.set_ylabel('Intensity (A.u.)')
            self.mplFourier.canvas.draw_idle()

            ## PHASE CALCULATION
            R = np.real(ft_data[:, 1])
            I = np.imag(ft_data[:, 1])

            # Calculating Phase
            phase = I / (R ** 2 + I ** 2) ** 0.5
            phase = sgf(phase, window_length=31, polyorder=3)

            # Normalization
            phase = phase/np.sqrt(np.sum(phase**2))

            # Plotting
            self.mplPhase.canvas.axes.clear()
            self.mplPhase.canvas.axes.plot(phase)
            self.mplPhase.canvas.axes.set_title("Phase")
            self.mplPhase.canvas.axes.set_xticks(np.linspace(0, len(df.columns), num_ticks))
            self.mplPhase.canvas.axes.set_xticklabels(map(int, np.linspace(last, first, num_ticks)))
            self.mplPhase.canvas.axes.set_xlabel('Time (s)')
            self.mplPhase.canvas.axes.set_ylabel('Phase')
            self.mplPhase.canvas.draw_idle()

            ## PHASE DERIVATIVE CALCULATION
            phase_deriv = np.diff(phase)
            phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

            # Normalization
            phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

            # Plotting
            self.mplPder.canvas.axes.clear()
            self.mplPder.canvas.axes.plot(phase_deriv)
            self.mplPder.canvas.axes.set_title("Phase Derivative")
            self.mplPder.canvas.axes.set_xlabel('Time (s)')
            self.mplPder.canvas.axes.set_xticks(np.linspace(0, len(df.columns), num_ticks))
            self.mplPder.canvas.axes.set_xticklabels(map(int, np.linspace(last, first, num_ticks)))
            self.mplPder.canvas.axes.set_ylabel('Phase Derivative')
            self.mplPder.canvas.draw_idle()

            # ========================== RESPONSE ACQUISITION ==========================

            ## STEADY STATE
            def steady_state(seq, n=60, eps=0.2):
                seq = abs(seq) < eps
                for i in range(len(seq)):
                    win = seq[i:i+n]
                    if sum(win) == n:
                        return int(i+n/2), i
                return 0, 0

            ss_pd = phase_deriv[:purge-20] # Use only the part of the spectrum when nitrogen is injected

            # Use half of one standard deviation as the steady state height condition (only applies to purge time)
            eps = min([np.mean(ss_pd) + np.std(ss_pd), np.mean(ss_pd) + np.std(ss_pd)])/2

            threshold = steady_state(ss_pd, eps=eps)[0]
            threshold_i = steady_state(ss_pd, eps=eps)[1]

            ## PEAKS AND TROUGHS
            ss_pd2 = phase_deriv[purge-20:] # Use only up until first odorant is injected
            eps2 = max([np.mean(ss_pd2) + np.std(ss_pd2), np.mean(ss_pd2) + np.std(ss_pd2)]) # take standard deviation of odorant part

            peak_list = [purge-20+i for i in list(peaks(ss_pd2, eps2))[0]]
            peak_height_vals = [i for i in peaks(ss_pd2, eps2)[1]['peak_heights']] # defining peak heights
            peak_height = [i for i in peak_height_vals if i in sorted(peak_height_vals, reverse=True)[:3]] # reverse to start with largest values
            peak_indices = [(i,j) for i,j in enumerate(peak_height_vals)]
            peak_indices.sort(key=lambda x: x[1], reverse=True)
            peak_list = [peak_list[i] for i in range(len(peak_list)) if i in [i[0] for i in peak_indices][:3]] # only take the highest 3 values

            trough_list = [purge-20+i for i in list(peaks(-ss_pd2, eps2))[0]]
            trough_height_vals = [i for i in peaks(-ss_pd2, eps2)[1]['peak_heights']]
            trough_height = [i for i in trough_height_vals if i in sorted(trough_height_vals, reverse=True)[:2]]
            trough_indices = [(i,j) for i,j in enumerate(trough_height_vals)]
            trough_indices.sort(key=lambda x:x[1], reverse=True)
            trough_list = [trough_list[i] for i in range(len(trough_list)) if i in [i[0] for i in trough_indices][:2]]

            dfcols = [float(i) for i in df.columns[:-1]] # we lose a single value by taking derivative of phase
            
            self.mplSteady.canvas.axes.clear()
            self.mplSteady.canvas.axes.plot(dfcols, phase_deriv)
            for i in peak_list: 
                self.mplSteady.canvas.axes.plot(float(df.columns[i]), phase_deriv[i], 'r*')
            for i in trough_list: 
                self.mplSteady.canvas.axes.plot(float(df.columns[i]), phase_deriv[i], 'g*')
            self.mplSteady.canvas.axes.plot(float(df.columns[threshold]), phase_deriv[threshold], 'k*')
            self.mplSteady.canvas.axes.set_title("Steady State")
            self.mplSteady.canvas.draw_idle()

            self.mplSteady.canvas.axes.axvspan(float(df.columns[threshold-30]),float(df.columns[threshold+30]), color="y", alpha=0.2) # plot steady state window
            self.mplSteady.canvas.axes.axhspan(-eps,eps, color="b", alpha=0.1) # plot steady state std window
            self.mplSteady.canvas.axes.axhspan(-eps2,eps2, color="b", alpha=0.1) # plot peak std window
            self.mplSteady.canvas.axes.vlines(float(df.columns[threshold]),min(phase_deriv), max(phase_deriv), alpha=0.5, linestyles={'dashed'}, label='{}'
                       .format((float(df.columns[threshold]), round(phase_deriv[threshold],2))))
            self.mplSteady.canvas.axes.legend()

            # -------------------------- SAVE RESPONSE VARIABLES TO SPREADSHEET -------------------------------
            to_df['Run'].append(run)
            try: 
                to_df['Steady State'].append(threshold)
                to_df['Peak 1'].append(peak_height[0])
                to_df['Peak 2'].append(peak_height[1])
                to_df['Peak 3'].append(peak_height[2])
                to_df['Trough 1'].append(trough_height[0])
                to_df['Trough 2'].append(trough_height[1])
            except IndexError:
                to_df['Peak 1'].append(0)
                to_df['Peak 2'].append(0)
                to_df['Peak 3'].append(0)
                to_df['Trough 1'].append(0)
                to_df['Trough 2'].append(0)

            df2 = pd.DataFrame(to_df)
            df2.set_index('Run', inplace=True)
            df2.head()
            df2.to_excel('../../Experiment Files/{}-results.xlsx'.format(exp_name))

            print("Saved all plots in ../Analysis Images/, and output {}-results.xlsx to ../Experiment Files/".format(exp_name))

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    experimentWindow = QtWidgets.QMainWindow()
    ui = Ui_experimentWindow()
    ui.setupUi(experimentWindow)
    experimentWindow.show()
    sys.exit(app.exec_())


138


SystemExit: 0

D:\Documents\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
